In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
import csv
import numpy as np
from bs4 import BeautifulSoup
import schedule
import time
from datetime import datetime
#Thư viện để dừng schedule
import sys
import os
import signal

#Cài đặt và chạy chrom
url = "https://iboard.ssi.com.vn/bang-gia/hose"
driver = webdriver.Chrome("E:\chromedriver_win32\chromedriver.exe")
driver.get(url)
#C:\Program Files (x86)\mingw-w64\i686-8.1.0-posix-dwarf-rt_v6-rev0\mingw32\opt\bin

#Đọc các mã chứng khoán trong file Ma_chung_khoan.txt
Ma_chung_khoan=open("Ma_chung_khoan.txt","r")
Ma_CK=Ma_chung_khoan.readlines()
Ma_chung_khoan.close() 
#loại bỏ dấu  ","
Ma_CK=Ma_CK[0].split(',')



In [2]:
#Xử lí lưu file
def xu_ly_file(a,mck):
        #Đường link file cào, sys.path[0] là đường dẫn thư mục của code         
        save_path = sys.path[0].replace('\\','/')+'/DATA_HOSE/'+mck
        #Kiểm tra xem đã có folder đó chưa, nếu chưa thì tạo mới
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        #Tên file.csv: Mã chứng khoán + ngày tháng năm
        completeName = os.path.join(save_path,mck+"_"+datetime.today().strftime('%d_%m_%Y')+".csv")
        #sửa mode w+ thành mode a để viết được vào cuối file
        file = open(completeName,"a",)
        with file:
            write = csv.writer(file)
            write.writerows(np.array(a).reshape(-1,27))
#Crawl Mã
def job():
    start_time = time.time()
    p=str(datetime.now())
    #download sourse html page
    page_source = BeautifulSoup(driver.page_source,"html.parser")
    for mck in Ma_CK:
        a=[]
        info_tr=page_source.find('tr', id=mck)
        info_td=info_tr.findAll('td')
        a.extend([p])
        for i in range(26):
            if info_td[i].get_text()=='':
                a.extend("N")
            else:
                a.append(info_td[i].get_text())
        xu_ly_file(a,mck)
    #print("--- %s seconds ---" % (time.time() - start_time))    
#Thoát file
def exit():
    os.kill(os.getpid(), signal.SIGTERM)
    os.exit()
    raise SystemExit("Stop right there!")


In [3]:
#hàm này để lồng được chạy từng giây
def anotherjob():
    schedule.every(1).seconds.do(job)
#hàm này để lồng được chạy từng giây
def anotherjob1():
    schedule.every(1).seconds.do(job)
#hàm này để lồng được chạy từng giây
def anotherjob2():
    schedule.every(1).seconds.do(job)
#hàm này để lồng được chạy từng giây
def anotherjob3():
    schedule.every(1).seconds.do(job)

#dòng này để chạy mỗi ngày vào giờ nào đó
schedule.every().day.at("21:56").do(anotherjob)
#anotherjob()
#dòng này để chạy mỗi ngày vào giờ nào đó
#schedule.every().day.at("21:06:59").do(anotherjob1)
#dòng này để chạy mỗi ngày vào giờ nào đó
#schedule.every().day.at("21:06:58").do(anotherjob2)
#dòng này để chạy mỗi ngày vào giờ nào đó
#schedule.every().day.at("21:06:57").do(anotherjob3)
schedule.every().day.at("21:57").do(exit)
#vòng while này kiểm tra mỗi giây coi đúng với tgian mình muốn để thực hiện hay chưa
while True:
    schedule.run_pending()
    time.sleep(1)


KeyboardInterrupt: 